In [19]:
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
import time
from astropy.time import Time
from astroquery.jplhorizons import Horizons
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 

originalDF = pd.read_csv('LempoRADEC.csv')

date = originalDF['time']
dateList =[]
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)
    
Lempo = Horizons(id='Lempo',location='500@0',epochs = dateList)

dist = Lempo.vectors()['range']
RA1 = Lempo.ephemerides()['RA']
DEC1 = Lempo.ephemerides()['DEC']

dRA2 = originalDF['x1']/3600
dDEC2 = originalDF['y1']/3600
dRA3 = originalDF['x2']/3600
dDEC3 = originalDF['y2']/3600

eRA2 = originalDF['ex1']/3600
eDEC2 = originalDF['ey1']/3600
eRA3 = originalDF['ex2']/3600
eDEC3 = originalDF['ey2']/3600

RA2 = np.array(RA1)+dRA2/np.cos(np.array(DEC1))
DEC2 = np.array(DEC1)+dDEC2

RA3 = np.array(RA1)+dRA3/np.cos(np.array(DEC1))
DEC3 = np.array(DEC1)+dDEC3

C1 = SkyCoord(ra=RA1, dec=DEC1, frame='icrs',distance = dist,unit=(u.deg,u.deg))
C2 = SkyCoord(ra=RA2*u.degree, dec=DEC2*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))
C3 = SkyCoord(ra=RA3*u.degree, dec=DEC3*u.degree, frame='icrs',distance = dist,unit=(u.deg,u.deg))

Ecl1 = C1.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
Ecl2 = C2.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
Ecl3 = C3.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))

Lat1 = Ecl1.lat.degree
Lon1 = Ecl1.lon.degree

Lat2 = Ecl2.lat.degree
Lon2 = Ecl2.lon.degree

Lat3 = Ecl3.lat.degree
Lon3 = Ecl3.lon.degree

dLat2 = Lat2-Lat1
dLon2 = (Lon2-Lon1)*np.cos(Lat1)

dLat3 = Lat3-Lat1
dLon3 = (Lon3-Lon1)*np.cos(Lat1)


ra2 = []
dec2 = []
ra3 = []
dec3 = []
c1 = []
c2 = []


for i in range(RA1.size):
    ra2.append(RA1[i] + np.random.normal(dRA2[i],eRA2[i],10000))
    dec2.append(DEC1[i] + np.random.normal(dDEC2[i],eDEC2[i],10000))
    ra3.append(RA1[i] + np.random.normal(dRA3[i],eRA3[i],10000))
    dec3.append(DEC1[i] + np.random.normal(dDEC3[i],eDEC3[i],10000))
    
for i in range(RA1.size):
    c1.append(SkyCoord(ra=ra2[i]*u.degree, dec=dec3[i]*u.degree, frame = 'icrs'))
    c2.append(SkyCoord(ra=ra3[i]*u.degree, dec=dec3[i]*u.degree, frame = 'icrs'))

ctrans1 = []
ctrans2 = []

for i in range(RA1.size):
    ctrans1.append(c1[i].transform_to('geocentrictrueecliptic'))
    ctrans2.append(c2[i].transform_to('geocentrictrueecliptic'))

eLat2 = []
eLon2 = []
eLat3 = []
eLon3 = []
for i in range(RA1.size):
    eLat2.append(np.std(ctrans1[i].lat.arcsec))
    eLon2.append(np.std(ctrans1[i].lon.arcsec))
    eLat3.append(np.std(ctrans2[i].lat.arcsec))
    eLon3.append(np.std(ctrans2[i].lon.arcsec))

latlondf = pd.DataFrame()
latlondf['time'] = date
latlondf['Lat1'] = Lat1
latlondf['Lon1'] = Lon1
latlondf['dLat2'] = dLat2*3600
latlondf['dLon2'] = dLon2*3600
latlondf['eLat2'] = eLat2
latlondf['eLon2'] = eLon2
latlondf['dLat3'] = dLat3*3600
latlondf['dLon3'] = dLon3*3600
latlondf['eLat3'] = eLat3
latlondf['eLon3'] = eLon3

latlondf.to_csv('Lempo_latlon_obs_df.csv')